<a href="https://colab.research.google.com/github/ialara/actf/blob/main/absorption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# F-16 Absorption Model
## Ignacio Lara

### Steps
1. Initialization
  1. Populate the squadron
    * Size (API-1 and API-6)
    * Distribution of INX/EXP
    * Distribution of sortie counts/qualifications/placement in upgrades
1. Inflow new pilots
  * Distribution of B-Course vs. FAIP/PQP (including qualifications of nth-tour)
  * Distribution of API-1 vs API-6
1. Determine resources available
   * Flying Hours remaining
   * Sorties (capacity, driven by UTE and PAA)
   * IPs and/or appropriate upgrade support pilots
1. Determine sortie needs
   * Upgrade vs. CT
   * ASD for FHP accrual
1. Schedule resources for sorties
1. Attempt to fly sorties
   * Refly rate to capture ground abort, incomplete sorties, wx cancellations, etc.
   * Decrement FHP for sorties flown
1. Credit sortie completion for syllabus and/or RTM "beans"
1. Award upgrades / experiencing
1. Outflow pilots

<hr />

### 0. Setup

In [1]:
a = {'foo': 3, 'bar': 247}

len(a)

2

In [2]:
import numpy as np
rng = np.random.default_rng()

ug_names = ['MQT', 'FLUG', 'IPUG']
ug_rides = [9,     9,      9]
### Simulator rides
# ug_sims =  [7, 5, 6]
###
ug_quals = ['WG',   'FL',   'IP']

fhp = 200
wss = 100

class Pilot:
    """An Air Force F-16 pilot that fills squadron billets"""
    # Syllabus lengths (number of UP sorties)
    syllabi_rides = {ug: dur for ug, dur in zip(ug_names, ug_rides)}

    # Quals awarded when completing upgrades
    ug_awards = {ug: q for ug, q in zip(ug_names, ug_quals)}

    # Everyone gets these many sorties from FTU
    ftu_sorties = 59 # Avg based on 49 WG PA release: https://bit.ly/3R8aADh

    # Definition of experience
    exp_sorties = 250
    exp_qual = ug_quals[1] # FL

    def __init__(self, id, f16_sorties, tos, api_category, quals=[], ug=None):
        self.id = id
        self.f16_sorties = self.ftu_sorties + f16_sorties
        self.tos = tos
        self.quals = quals
        self.is_exp = False
        self.check_experience()
        self.api_category = 1 if not self.is_exp else api_category
        self.ug = None
        if ug is not None:
            self.enroll_upgrade(ug)

    def log(self, msg, prefix='>>'):
        print(f'{prefix} PID {self.id} {msg}')

    def enroll_upgrade(self, ug):
        assert self.ug is None, 'Pilot already enrolled in upgrade'
        assert ug in self.ug_awards, 'Invalid upgrade specified'
        assert self.ug_awards[ug] not in self.quals, 'Pilot already completed upgrade'
        self.ug = ug
        self.ride_num = 0
        self.pending_qual = self.ug_awards[self.ug]
        self.log(f'enrolled in {self.ug}')

    def disenroll_upgrade(self):
        prev_ug = self.ug
        self.ug = None
        self.log(f'disenrolled from: {prev_ug}', prefix='<<')

    def fly_ug_sortie(self):
        assert self.ug is not None, 'Pilot not enrolled in upgrade'
        self.ride_num += 1
        if self.check_ug_complete(self.ug, self.ride_num):
          self.award_qual(self.pending_qual)
          self.disenroll_upgrade()
        self.fly_sortie()

    def check_ug_complete(self, ug, ride):
        return ride == self.syllabi_rides[ug]

    def fly_sortie(self):
        self.f16_sorties += 1
        self.check_experience()

    def award_qual(self, qual):
        assert qual in self.ug_awards.values(), 'Invalid qualification specified'
        assert qual not in self.quals, 'Pilot already qualified'
        self.quals.append(qual)
        self.log(f'awarded: {qual}', prefix='++')
        self.check_experience()

    def remove_qual(self, qual):
        assert qual in self.quals, 'Pilot does not have this qualification'
        self.quals.remove(qual)
        self.log(f'un-awarded: {qual}', prefix='--')
        self.check_experience()

    def check_experience(self):
        prev_status = self.is_exp
        self.is_exp = (self.f16_sorties >= self.exp_sorties and 
                      self.exp_qual in self.quals)
        if self.is_exp and not prev_status:
            self.log('EXPERIENCED', prefix='**')

    def increment_tos(self, months=1):
      assert isinstance(months, (int, float)), 'Months must be numeric'
      self.tos += months

    def return_experience(self):
      return self.is_exp

    def return_upgrade(self):
      return self.ug

    def return_api(self):
      return self.api_category

    def return_highest_qual(self):
      if self.quals == []:
        return ''
  
      return self.quals[-1]

    def return_tos(self):
      return self.tos

    def summarize(self):
        text = 'PID {:2d}: {}{} | SOR: {:4d} | TOS: {:04.1f} mo. | QL: {}'.format(self.id,
                                                            '6-' if self.api_category == 6 else '',
                                                            'EXP' if self.is_exp else 'INX',
                                                            self.f16_sorties,
                                                            self.tos,
                                                            self.quals)
        if self.ug is not None:
            text += f' | UG: {self.ug} #{self.ride_num + 1}' # +1 to show next (pending) ride
            if self.ride_num == self.syllabi_rides[self.ug] - 1:
              text += ' (CERT)'
        return text

class SquadronRoster:
    """A collection of Air Force pilots in the same squadron"""
    
    def __init__(self, sq_name, pilots={}):
        self.sq_name = sq_name
        self.pilots = dict(pilots)
        self.pid = 0

    def log(self, msg, prefix='=='):
        print(f'{prefix} {self.sq_name} {msg}')

    def add_INX_pilot(self, f16_sorties=0, tos=0, quals=[], ug=None):
        pid = self.next_pid()
        self.pilots[pid] = Pilot(pid, f16_sorties, tos, api_category=1, quals=quals, ug=ug)
        return self.pilots[pid]

    def next_pid(self):
        if len(self.pilots) > 0:
          self.pid = max(self.pid, max(self.pilots)) + 1
        return self.pid

    def add_EXP_pilot(self, f16_sorties=250, tos=20, api_category=1, quals=ug_quals[:-1], ug=None):
        pid = self.next_pid()
        self.pilots[pid] = Pilot(pid, f16_sorties, tos, api_category, quals, ug)
        return self.pilots[pid]

    def add_pilots(self, new_pilots):
        assert all(isinstance(p, Pilot) for p in new_pilots.values()), 'List must contain Pilot elements'
        self.pilots = self.pilots | new_pilots
        new_pids = new_pilots.keys()
        self.log(f'added PIDs {new_pids}')
        return new_pilots

    def remove_pilots(self, pids):
        assert all(pid in self.pilots for pid in pids), 'Pilot not found'
        self.log(f'removed PIDs {pids}', prefix='..')
        return [self.pilots.pop(pid) for pid in pids] 

    def populate(self, num_API1=26, prop_EXP=0.45, prop_IP=0.4, num_API6=10):
        num_EXP = rng.binomial(num_API1, prop_EXP)
        num_INX = num_API1 - num_EXP
        ip_billets_remaining = rng.binomial(num_EXP, prop_IP)
        staff_billets_remaining = num_API6

        # Parameters
        max_TOS_INX = 24
        min_TOS_EXP = 24
        max_TOS_EXP = 32

        min_f16_sorties_EXP = Pilot.exp_sorties
        max_f16_sorties_EXP = 500

        max_f16_sorties_INX = Pilot.exp_sorties

        staff_billets_remaining = num_API6
        # Add EXP pilots
        for _ in range(num_EXP + num_API6):
            tos = rng.integers(min_TOS_EXP, max_TOS_EXP + 1)
            # Award sorties proportional to TOS
            sorties = int((tos - min_TOS_EXP)/(max_TOS_EXP - min_TOS_EXP) *
                      (max_f16_sorties_EXP - min_f16_sorties_EXP)) + min_f16_sorties_EXP

            quals = ug_quals[:-1] # Assumed that all EXP are at least FL

            if staff_billets_remaining > 0:
                api = 6 # Assign staff first
                staff_billets_remaining -= 1
            else:
                api = 1
                # Assume only API-1 arrive as IP
                if ip_billets_remaining > 0:
                    quals.append(ug_quals[-1])
                    ip_billets_remaining -= 1
              
            self.add_EXP_pilot(sorties, tos, api, quals)

        # Add INX pilots
        for _ in range(num_INX):
            tos = rng.integers(max_TOS_INX + 1)
            sorties = int(tos/max_TOS_INX * max_f16_sorties_INX)

            quals = []

            pilot = self.add_INX_pilot(sorties, tos, quals=[])

            if pilot.tos > 2:
              pilot.award_qual(ug_quals[0])

            else:
              mqt_progress = rng.integers(pilot.syllabi_rides[ug_names[0]]) # Intentionally capping starting INX at 1 MQT ride from end
              pilot.enroll_upgrade(ug_names[0])
              for s in range(mqt_progress):
                pilot.fly_ug_sortie()

    def inflow_nth_tour(self, num_API1, prop_API1_IP, prop_WG, num_API6, prop_API6_IP):
      num_IP = rng.binomial(num_API1, prop_API1_IP)
      num_INX = rng.binomial(num_API1, prop_WG)
      print(f'IP: {num_IP} | INX: {num_INX}')
      num_FL = num_API1 - num_IP - num_INX

      min_sorties_INX = int(0.6*Pilot.exp_sorties)
      max_sorties_INX = int(1.1*Pilot.exp_sorties)

      min_sorties_FL = Pilot.exp_sorties
      max_sorties_FL = int(2.5*Pilot.exp_sorties)

      min_sorties_IP = int(1.5*Pilot.exp_sorties)
      max_sorties_IP = 5*Pilot.exp_sorties

      new_nth_pilots = []

      for _ in range(num_INX):
        sorties = rng.integers(min_sorties_INX, max_sorties_INX)
        pilot = self.add_INX_pilot(f16_sorties=sorties, quals=[ug_quals[0]])
        new_nth_pilots.append(pilot)

      for _ in range(num_FL):
        sorties = rng.integers(min_sorties_FL, max_sorties_FL)
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, quals=ug_quals[:2])
        new_nth_pilots.append(pilot)

      num_API6_IP = rng.binomial(num_API6, prop_API6_IP)
      for _ in range(num_API6):
        sorties = rng.integers(min_sorties_FL, max_sorties_IP)
        my_quals = ug_quals[:2]

        if num_API6_IP > 0:
          my_quals += [ug_quals[-1]]
          num_API6_IP -= 1
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, api_category=6, quals=my_quals)
        new_nth_pilots.append(pilot)

      for _ in range(num_IP):
        sorties = rng.integers(min_sorties_IP, max_sorties_IP)
        pilot = self.add_EXP_pilot(f16_sorties=sorties, tos=0, quals=ug_quals)
        new_nth_pilots.append(pilot)

      return new_nth_pilots

    def inflow_first_tour(self, num_API1):
      new_pilots = []

      for _ in range(num_API1):
        ftu_sortie_delta = rng.integers(-5, 11)
        pilot = self.add_INX_pilot(f16_sorties=ftu_sortie_delta, quals=[])
        new_pilots.append(pilot)

      return new_pilots

    def outflow_pilots(self):
      tos_cap = 32
      pids_tos_exceeded = [id for id in self.pilots 
                           if self.pilots[id].return_tos() >= tos_cap]
      self.remove_pilots(pids_tos_exceeded)

    def get_pilots_qualified_as(self, qual):
      return {id: pilot for id, pilot in self.pilots.items() if pilot.return_highest_qual() == qual}

    def get_students_in(self, ug):
      return {id: pilot for id, pilot in self.pilots.items() if pilot.return_upgrade() == ug}

    def summarize(self):
      roster_API1 = {k: p for k, p in self.pilots.items() if p.return_api() == 1}

      num_EXP = np.sum([p.return_experience() for p in roster_API1.values()])
      num_INX = len(roster_API1) - num_EXP
      exp_pct = num_EXP / (num_EXP + num_INX)
      
      upgrades = [p.return_upgrade() for p in roster_API1.values()]
      highest_quals = [p.return_highest_qual() for p in roster_API1.values()]
      num_MQT = upgrades.count(ug_names[0])
      num_FLUG = upgrades.count(ug_names[1])
      num_IPUG = upgrades.count(ug_names[2])
      num_WG = highest_quals.count(ug_quals[0])
      num_FL = highest_quals.count(ug_quals[1])
      num_IP = highest_quals.count(ug_quals[2])

      print(f'{self.sq_name} SUMMARY:')
      print(
          f'API-1 ({len(roster_API1)}) >> '
          f'EXP: {num_EXP} / INX: {num_INX} | EXP% (API-1): {exp_pct:.2f} | '
          f'API-6: {len(self.pilots)-len(roster_API1)} | '
          f'{ug_quals[0]}: {num_WG} (+{num_MQT} in {ug_names[0]}) / '
          f'{ug_quals[1]}: {num_FL} (+{num_FLUG} in {ug_names[1]}) / '
          f'{ug_quals[2]}: {num_IP} (+{num_IPUG} in {ug_names[2]}) ')

    
    def print_sq(self):
        print(f'{self.sq_name} ROSTER:')
        for p in self.pilots.values():
          print(p.summarize())

class FlySchedule:
  ug_priorities = {'MQT': 1,
                   'IPUG': 2,
                   'FLUG': 3}

  scheduling_philosophy = 'progression'

  max_fhp = 80
  max_wss = 68

  def __init__(self, students, instructors, support, gos=3, name='default_FlySchedule'):
    self.name = name
    self.ups = students
    self.ips = instructors
    self.spt = support
    self.gos = gos

  def assign_student_priorities(self, students):
    for id, student in students.items():
      student['priority'] = FlySchedule.ug_priorities[student.return_upgrade()]

  def make_schedule(self):
    scheduled_lines = []
    ips = {}
    ups = {}
    spt = {}

    for _ in range(self.gos):
      available_students = list(self.ups)
      if self.ug_sortie_possible(self.ips, self.ups, self.spt):
        ip = self.ips[rng.choice(list(self.ips))]
        up = self.ups[rng.choice(available_students)]
        available_students.remove(up.id) # Student can't fly twice on same day
        num_spt = 2
      else:
        num_spt = 4 # CT
      
      if len(self.spt) >= num_spt:
        spt_ids = rng.choice(list(self.spt), num_spt, replace=False)
        spt = [self.spt[id] for id in spt_ids]
      
      scheduled_lines.append(FlightCrew(ips, ups, spt))
      return scheduled_lines    

  def ug_sortie_possible(self, ips, students, support):
    return len(ips) > 0 and len(students) > 0 and len(support) >= 2  

class FlightCrew:
  def __init__(self, ips={}, ups={}, spt={}):
    labels = ['IP', 'UP', 'Support']
    self.crew = {lab: None for lab in labels}

    for duty_position, pilots in zip(labels,[ips, ups, spt]):
      print(pilots)
      if len(pilots) > 0:
        self.crew[duty_position] = pilots

  def summarize(self):
    return {duty: [p.id for p in pilot] for duty, pilot in self.crew.items() if pilot is not None}

class SimMonth:
  month_num = 0
  def __init__(self, roster):
    self.month = SimMonth.month_num
    self.roster = roster

  def advance(self, months=1):
    SimMonth.month_num += 1
    print(f'=====Month {SimMonth.month_num}====')
    self.update_tos_values(self.roster.pilots, months)
    self.do_inflows(self.roster)
    self.do_outflows(self.roster)
    for _ in range(20):
      self.fly_day(self.roster)

  def update_tos_values(self, pilots, months):
    for id, pilot in pilots.items():
      pilot.increment_tos(months)

  def do_inflows(self, roster):
    self.roster.inflow_nth_tour(num_API1=10, prop_API1_IP=.4, prop_WG=.2, 
                                num_API6=3, prop_API6_IP=.6)
    self.roster.inflow_first_tour(8)

  def do_outflows(self, roster):
    self.roster.outflow_pilots()

  def fly_day(self, roster):
    print('---New day---')
    ips = roster.get_pilots_qualified_as('IP')
    ups = roster.get_students_in('MQT')
    spt = roster.get_pilots_qualified_as('FL')
    sched = FlySchedule(ups, ips, spt)
    lines = sched.make_schedule()
    for line in lines:
      print(line.summarize())
      self.update_instr_sorties(line.crew['IP'])
      self.update_ug_sorties(line.crew['UP'])
      self.update_spt_sorties(line.crew['Support'])

  def update_instr_sorties(self, ip):
    if ip is not None:
      ip.fly_sortie()

  def update_ug_sorties(self, up):
    if up is not None:
      up.fly_ug_sortie()

  def update_spt_sorties(self, spt):
    if spt is not None:
      for pilot in spt:
        pilot.fly_sortie()

  


# class F16:
#   """A single F-16 jet"""
#   hours_between_depot = 500
#   depot_days = 90
#   mc_rate = 0.7

#   def __init__(self, id, num_seats=1, ttaf=2000,
#                hours_since_last_depot=500):
#     self.id = id
#     self.num_seats = num_seats
#     self.mc_random_draw()
#     self.ttaf = ttaf
#     self.hours_since_last_depot = hours_since_last_depot

#   def log(self, msg, prefix='>>'):
#     print(f'{prefix} {self.id} {msg}')

#   def set_nmc(self):
#     self.log('is NMC', prefix='<<')
#     self.is_mc = False

#   def set_mc(self):
#     self.log('is MC')
#     self.is_mc = True

#   def mc_random_draw(self):
#     self.is_mc = rng.random() <= self.mc_rate

#   def fly_hours(self, hours):
#     assert self.is_mc, "Cannot fly NMC aircraft"
#     self.ttaf += hours
#     self.hours_since_last_depot += hours
#     self.log(f'flew {hours} hours. TTAF: {self.ttaf}. Hrs since depot: {self.hours_since_last_depot}', prefix='++')
#     if self.needs_depot():
#       self.start_depot()

#   def sortie(self, duration):
#     if self.is_mc:
#       self.fly_hours(duration)
#     else:
#       self.log('not flown due to NMC.', prefix='<X>')

#   def needs_depot(self):
#     return self.hours_since_last_depot >= self.hours_between_depot

#   def start_depot(self):
#     self.log('sent to depot!', prefix='!!')
#     self.set_nmc()
#     self.days_left_in_depot = self.depot_days

#   def sit_in_depot(self):
#     assert self.days_left_in_depot > 0, "Aircraft already overdue from depot"
#     self.days_left_in_depot -= 1

#   def complete_depot(self):
#     self.log('depot complete.', prefix='**')
#     self.set_mc()
#     self.days_left_in_depot = -1
#     self.hours_since_last_depot = 0

#   def print_ac(self):
#     print(f'{self.id} | MC?: {self.is_mc} | TTAF: {self.ttaf} | Hrs. since depot: {self.hours_since_last_depot}')

# class SquadronHangar:
#   """A collection of F-16s assigned to a squadron"""
#   def __init__(self, jets=[]):
#     pass

#   def populate(self):
#     pass

#   def summarize(self):
#     pass

#   def make_fly_order(self):
#     pass


# class DailySchedule:
#   """Pairing of crews and aircraft """
#   pass






## Pilot class example

Contains methods and properties associated with individual pilots. For example:

_Methods_
- Fly a sortie
- Enroll in/disenroll from upgrade
- Become experienced

_Properties_
- \# sorties
- Time on Station
- API category
- Inexperienced/Experienced status
- Qualifications
- Upgrade progress

Class methods also contain safeguards against improper usage. In this example, we first create an `example_pilot`, with `pid = 0` (pilot ID), `f16_sorties = 24`, `tos = 4` months, and an `API=1`, with no qualifications. Then, we:
1. Fly a single sortie for the pilot, updating total sortie count only.
1. Attempt to fly an upgrade sortie, even though the pilot is not enrolled in an upgrade (will raise an `AssertionError`). 

In [4]:
example_pilot = Pilot(id=0, f16_sorties=24, tos=4, api_category=1, quals=[])
print(example_pilot.summarize())

example_pilot.fly_sortie()
print('---After flying sortie---')
print(example_pilot.summarize())
print('---Attempt to fly upgrade sortie with no upgrade---')
example_pilot.fly_ug_sortie()

PID  0: INX | SOR:   83 | TOS: 04.0 mo. | QL: []
---After flying sortie---
PID  0: INX | SOR:   84 | TOS: 04.0 mo. | QL: []
---Attempt to fly upgrade sortie with no upgrade---


AssertionError: ignored

We fix the `AssertionError` by enrolling the pilot in MQT. Now, it makes sense to fly an upgrade sortie, which updates totals like before, as well as progressing the pilot through an upgrade syllabus and checking for completion.
1. The pilot's progress through MQT is indicated by `UG: MQT #0` (only when enrolled in an upgrade). When the pilot completes MQT, the pilot is awarded the Wingman qualification, shown in `QL = ['WG']`, and disenrolled from MQT.
1. When the pilot meets 250 sorties, but without completing FLUG, the pilot is still recognized as inexperienced (`INX`).
1. After the pilot completes FLUG, and meets the sortie minimum, the pilot is awarded the FL qualification, disenrolled from FLUG, and updated to reflect experienced status (`EXP`).

In practice, these methods and properties will be called by higher-level functions in the simulation. But this example demonstrates some of the functionality present in the `Pilot` class.

In [5]:
example_pilot.enroll_upgrade('MQT')
print(example_pilot.summarize())

example_pilot.fly_ug_sortie()
print('---After flying MQT sortie---')
print(example_pilot.summarize())

>> PID 0 enrolled in MQT
PID  0: INX | SOR:   84 | TOS: 04.0 mo. | QL: [] | UG: MQT #1
---After flying MQT sortie---
PID  0: INX | SOR:   85 | TOS: 04.0 mo. | QL: [] | UG: MQT #2


In [6]:
print('---Fly remaining MQT sorties---')
for _ in range(example_pilot.syllabi_rides[example_pilot.ug] - example_pilot.ride_num):
  example_pilot.fly_ug_sortie()
print(example_pilot.summarize())
print()

example_pilot.f16_sorties = 250
print('---Meet EXP sortie criteria, but not FL---')
print(example_pilot.summarize())
print()

example_pilot.enroll_upgrade('FLUG')

for _ in range(example_pilot.syllabi_rides[example_pilot.ug] - 1 - example_pilot.ride_num):
  example_pilot.fly_ug_sortie()

print('---One ride from completing FLUG---')
print(example_pilot.summarize())
print()

example_pilot.fly_ug_sortie()
print('---After completing FLUG, now meets criteria for EXP---')
print(example_pilot.summarize())


---Fly remaining MQT sorties---
++ PID 0 awarded: WG
<< PID 0 disenrolled from: MQT
PID  0: INX | SOR:   93 | TOS: 04.0 mo. | QL: ['WG']

---Meet EXP sortie criteria, but not FL---
PID  0: INX | SOR:  250 | TOS: 04.0 mo. | QL: ['WG']

>> PID 0 enrolled in FLUG
---One ride from completing FLUG---
PID  0: INX | SOR:  258 | TOS: 04.0 mo. | QL: ['WG'] | UG: FLUG #9 (CERT)

++ PID 0 awarded: FL
** PID 0 EXPERIENCED
<< PID 0 disenrolled from: FLUG
---After completing FLUG, now meets criteria for EXP---
PID  0: EXP | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


In [7]:
example_pilot.increment_tos()
print(example_pilot.summarize())

example_pilot.increment_tos(1.5)
print(example_pilot.summarize())

example_pilot.increment_tos(-2.5)
print(example_pilot.summarize())

PID  0: EXP | SOR:  259 | TOS: 05.0 mo. | QL: ['WG', 'FL']
PID  0: EXP | SOR:  259 | TOS: 06.5 mo. | QL: ['WG', 'FL']
PID  0: EXP | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


In [8]:
example_pilot.remove_qual('FL')
print(example_pilot.summarize())

example_pilot.award_qual('FL')
print(example_pilot.summarize())

-- PID 0 un-awarded: FL
PID  0: INX | SOR:  259 | TOS: 04.0 mo. | QL: ['WG']
++ PID 0 awarded: FL
** PID 0 EXPERIENCED
PID  0: EXP | SOR:  259 | TOS: 04.0 mo. | QL: ['WG', 'FL']


## SquadronRoster example

Contains the Pilots (API-1 and API-6) present in a single F-16 squadron. Has some methods and properties that make sense at the squadron level. For example:

_Methods_:
- Add/remove individual Pilots
- Populate an initial squadron with desired size, EXP%, and qualifications

_Properties_:
- Squadron name
- Squadron summary
- Summary statistics (EXP%, upgrade sizes, etc.)

In this example, we create a notional squadron.

The defaults for `SquadronRoster.populate` include:
- `num_API1=26` API-1 pilots
- `prop_EXP=0.45` percent of API-1 pilots experienced
- `prop_IP=0.4` percent of experienced API-1 pilots qualified as instructors
- `num_API6=10` API-6 staff pilots

In [9]:
sq = SquadronRoster('0th FS', pilots=[])
sq.populate()
sq.print_sq()
print()
sq.summarize()

** PID 0 EXPERIENCED
** PID 1 EXPERIENCED
** PID 2 EXPERIENCED
** PID 3 EXPERIENCED
** PID 4 EXPERIENCED
** PID 5 EXPERIENCED
** PID 6 EXPERIENCED
** PID 7 EXPERIENCED
** PID 8 EXPERIENCED
** PID 9 EXPERIENCED
** PID 10 EXPERIENCED
** PID 11 EXPERIENCED
** PID 12 EXPERIENCED
** PID 13 EXPERIENCED
** PID 14 EXPERIENCED
** PID 15 EXPERIENCED
** PID 16 EXPERIENCED
** PID 17 EXPERIENCED
** PID 18 EXPERIENCED
** PID 19 EXPERIENCED
** PID 20 EXPERIENCED
** PID 21 EXPERIENCED
** PID 22 EXPERIENCED
** PID 23 EXPERIENCED
** PID 24 EXPERIENCED
** PID 25 EXPERIENCED
++ PID 26 awarded: WG
++ PID 27 awarded: WG
++ PID 28 awarded: WG
>> PID 29 enrolled in MQT
++ PID 30 awarded: WG
++ PID 31 awarded: WG
++ PID 32 awarded: WG
++ PID 33 awarded: WG
++ PID 34 awarded: WG
++ PID 35 awarded: WG
0th FS ROSTER:
PID  0: 6-EXP | SOR:  371 | TOS: 26.0 mo. | QL: ['WG', 'FL']
PID  1: 6-EXP | SOR:  496 | TOS: 30.0 mo. | QL: ['WG', 'FL']
PID  2: 6-EXP | SOR:  527 | TOS: 31.0 mo. | QL: ['WG', 'FL']
PID  3: 6-EXP | 

## SimMonth example

In [10]:
sq.summarize()

0th FS SUMMARY:
API-1 (26) >> EXP: 16 / INX: 10 | EXP% (API-1): 0.62 | API-6: 10 | WG: 9 (+1 in MQT) / FL: 9 (+0 in FLUG) / IP: 7 (+0 in IPUG) 


In [11]:
my_month = SimMonth(sq)

my_month.advance()

=====Month 1====
IP: 6 | INX: 0
** PID 36 EXPERIENCED
** PID 37 EXPERIENCED
** PID 38 EXPERIENCED
** PID 39 EXPERIENCED
** PID 40 EXPERIENCED
** PID 41 EXPERIENCED
** PID 42 EXPERIENCED
** PID 43 EXPERIENCED
** PID 44 EXPERIENCED
** PID 45 EXPERIENCED
** PID 46 EXPERIENCED
** PID 47 EXPERIENCED
** PID 48 EXPERIENCED
.. 0th FS removed PIDs [2, 5, 9, 12, 13, 21, 25]
---New day---
{}
{}
[<__main__.Pilot object at 0x7f285d237f50>, <__main__.Pilot object at 0x7f285d241110>]
{'Support': [20, 7]}
---New day---
{}
{}
[<__main__.Pilot object at 0x7f285d241110>, <__main__.Pilot object at 0x7f285d256d10>]
{'Support': [7, 6]}
---New day---
{}
{}
[<__main__.Pilot object at 0x7f285d2373d0>, <__main__.Pilot object at 0x7f2861494d50>]
{'Support': [8, 37]}
---New day---
{}
{}
[<__main__.Pilot object at 0x7f2861494c50>, <__main__.Pilot object at 0x7f285d2567d0>]
{'Support': [36, 4]}
---New day---
{}
{}
[<__main__.Pilot object at 0x7f285d22c350>, <__main__.Pilot object at 0x7f285d22ccd0>]
{'Support': [17

In [ ]:
type(sq)

In [ ]:
sq.summarize()

In [ ]:
ips = sq.get_pilots_qualified_as('IP')
ups = sq.get_students_in('MQT')
spt = sq.get_pilots_qualified_as('FL')

In [ ]:
sched = FlySchedule(ips, ups, spt)
crews = sched.make_schedule()

[crew.summarize() for crew in crews]

d## 1. Inflow Pilots

In [ ]:
new_pilots = sq.inflow_nth_tour(num_API1=10, prop_API1_IP=.4, prop_WG=.2, 
                                num_API6=5, prop_API6_IP=.6)

sq.summarize()


In [ ]:
[p.summarize() for p in new_pilots]

In [ ]:
new_pilots = sq.inflow_first_tour(8)
sq.summarize()

In [ ]:
[p.summarize() for p in new_pilots]

In [ ]:
sq.print_sq()

In [ ]:
sq.summarize()

## 2. Outflow pilots

`outflow_pilots` will remove any pilots with >= 32 months on station:

In [ ]:
sq.print_sq()
print()
sq.outflow_pilots()

sq.print_sq()